In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [25]:
import os
import pandas as pd
import glob
import pickle

os.sys.path.insert(0, '../evaluation')

from evaluate import evaluate_all_systems, preprocess_all_models

In [26]:
models = [os.path.basename(p) for p in glob.glob(f'../data/models/dev/*')]

In [27]:
dfs = []
for model in models:
    if os.path.isfile(f'../data/models/dev/{model}/system_evaluation.csv'):
        df_ = pd.read_csv(f'../data/models/dev/{model}/system_evaluation.csv', index_col=['subset', 'references', 'metric'])
        dfs.append(df_)

scores_df = pd.concat(dfs, keys=models).unstack()
scores_df.columns = scores_df.columns.droplevel()

In [28]:
params_dfs = []
for model in models:
    with open(f'../data/models/dev/{model}/params.pkl', 'rb') as f:
        params_dfs.append(pd.DataFrame([pickle.load(f)], index=[model]))
    
params_df = pd.concat(params_dfs)
params_df.tems_lm_n.fillna(0, inplace=True)
params_df.txs_lm_n.fillna(0, inplace=True)

In [29]:
elapsed_time_dfs = []
for model in models:
    with open(f'../data/models/dev/{model}/elapsed_time.txt', 'r') as f:
        elapsed_time_dfs.append(pd.DataFrame(data=[float(f.readline()[:-1])], index=[model], columns=['elapsed_time']))
elapsed_time_df = pd.concat(elapsed_time_dfs)

In [30]:
scores_all = scores_df.loc[(slice(None), 'all-cat', slice(None)), :].reset_index(level=[1, 2], drop=True)

df_all = pd.merge(scores_all, params_df, left_index=True, right_index=True)
df_all = pd.merge(df_all, elapsed_time_df, left_index=True, right_index=True)

cols = ['bleu', 'meteor', 'ter', 'elapsed_time', 'dp_scorer_n', 'sa_scorer_n', 'tems_lm_n', 'referrer_lm_n', 'txs_lm_n', 'max_dp', 'max_sa', 'max_refs', 'max_tems']
df_all = df_all[cols]
df_all.sort_values(by='bleu', ascending=False)

,bleu,meteor,ter,elapsed_time,dp_scorer_n,sa_scorer_n,tems_lm_n,referrer_lm_n,txs_lm_n,max_dp,max_sa,max_refs,max_tems
8427712258203685535,60.67,0.439775,0.350059,25.549334,2,3,4,4,3,5,5,1,2
-6120210850613917316,60.65,0.440085,0.349864,26.118550,2,2,4,4,3,5,5,1,2
8608132448910299145,60.53,0.439110,0.350254,25.506509,2,2,4,3,3,5,5,1,2
2528891278422703928,60.46,0.441876,0.356933,7.248840,2,2,4,4,4,5,5,1,1
4901813669408019549,60.43,0.439895,0.354691,7.174427,2,2,4,4,3,5,5,1,1
-2785595828672947740,60.42,0.442309,0.350449,25.454383,2,2,4,3,4,5,5,1,2
-1922451055525070619,60.41,0.441268,0.357128,7.072687,2,2,4,3,4,5,5,1,1
-468997385542151278,60.39,0.438812,0.350790,24.174220,2,3,4,3,3,5,5,1,2
6316591217736520224,60.33,0.439481,0.353569,7.214782,2,2,4,3,3,5,5,1,1
3106577197184455726,60.31,0.441738,0.351570,7.411245,2,3,4,3,4,5,5,1,1


In [15]:
df_all.groupby(['dp_scorer_n', 'sa_scorer_n', 'tems_lm_n', 'txs_lm_n']).bleu.describe().sort_values('75%', ascending=False)

count       mean       std    min  \
dp_scorer_n sa_scorer_n tems_lm_n txs_lm_n                                      
2           2           4         4           3.0  60.153333  0.488501  59.59   
            3           4         4           2.0  60.270000  0.056569  60.23   
                                  3           1.0  60.260000       NaN  60.26   
            2           3         3           3.0  59.906667  0.594755  59.22   
                        4         3           3.0  60.030000  0.310483  59.71   
            3           3         4           2.0  59.875000  0.784889  59.32   
                                  3           3.0  59.653333  0.759232  59.21   
            2           3         4           3.0  59.690000  0.480729  59.35   

                                                25%     50%      75%    max  
dp_scorer_n sa_scorer_n tems_lm_n txs_lm_n                                   
2           2           4         4         60.0000  60.410  60.4350  60.46  
            3           4         4         60.2500  60.270  60.2900  60.31  
                                  3         60.2600  60.260  60.2600  60.26  
            2           3         3         59.7300  60.240  60.2500  60.26  
                        4         3         59.8800  60.050  60.1900  60.33  
            3           3         4         59.5975  59.875  60.1525  60.43  
                                  3         59.2150  59.220  59.8750  60.53  
            2           3         4         59.4150  59.480  59.8600  60.24

In [14]:
df_all[(df_all.txs_lm_n == 3) & (df_all.txs_lm_name == 'lower') & (df_all.tems_lm_name == 'lower') & (df_all.tems_lm_n == 3) & (df_all.sa_scorer == 'lower')].sort_values('bleu', ascending=False)

,bleu,meteor,ter,dp_scorer,max_dp,sa_scorer,max_sa,tems_lm_name,tems_lm_n,max_tems,referrer,txs_lm_name,txs_lm_n


In [55]:
df_all.columns = ['BLEU', 'METEOR', 'TER', 'MPerm', 'n_MPerm', 'MPart', 'n_MPart', 'MTemp', 'MTemp_N', 'n_MTemp', 'MREG', 'MTRank', 'MTRank_N']

In [58]:
df_all = df_all[df_all.MPerm != 'gold']
df_all.to_csv('../data/results/dev_all_results.csv')

In [49]:
scores_5 = scores_df.loc[(slice(None), '5size', slice(None)), :].reset_index(level=[1, 2], drop=True)

df_5 = pd.merge(scores_5, params_df, left_index=True, right_index=True)

cols = ['bleu', 'meteor', 'ter', 'dp_scorer', 'max_dp', 'sa_scorer', 'max_sa', 'tems_lm_name', 'tems_lm_n', 'max_tems', 'referrer', 'txs_lm_name', 'txs_lm_n']
df_5.sort_values('bleu', ascending=False).loc[:, cols].head()

,bleu,meteor,ter,dp_scorer,max_dp,sa_scorer,max_sa,tems_lm_name,tems_lm_n,max_tems,referrer,txs_lm_name,txs_lm_n
37677081755143658,52.81,NaN,NaN,ltr_lasso,2,inv_ltr_lasso,4,lower,3,2,counter,lower,3
4692801175971517865,52.69,NaN,NaN,ltr_lasso,2,inv_ltr_lasso,4,lower,3,2,counter,lower,6
7697843703963551123,52.28,NaN,NaN,ltr_lasso,2,inv_ltr_lasso,4,lower,6,2,counter,lower,3
-5304774152169464302,51.63,NaN,NaN,gold,2,gold,4,lower,3,2,counter,lower,3
-9198725233750128576,51.63,NaN,NaN,ltr_lasso,2,inv_ltr_lasso,4,lower,6,2,counter,lower,6


In [50]:
df_5 = df_5[df_5.dp_scorer != 'gold']
df_5.to_csv('../data/results/dev_5_results.csv')

# Groupby module bleu

## all-cat

In [57]:
df_all.groupby('dp_scorer').bleu.describe()

,count,mean,std,min,25%,50%,75%,max
dp_scorer,,,,,,,,
ltr_lasso,104.0,39.655577,10.435262,17.74,27.6975,40.785,43.9125,58.18


In [58]:
df_all.groupby('sa_scorer').bleu.describe()

,count,mean,std,min,25%,50%,75%,max
sa_scorer,,,,,,,,
inv_ltr_lasso,32.0,39.498125,8.770645,25.11,35.87,40.435,42.8575,57.81
ltr_lasso,41.0,40.020244,11.269731,17.80,27.66,41.560,49.6800,57.77
random,31.0,39.335806,11.168130,17.74,27.45,40.750,43.7400,58.18


In [59]:
df_all.groupby(['tems_lm_name', 'tems_lm_n']).bleu.describe()

count       mean        std    min      25%     50%  \
tems_lm_name tems_lm_n                                                        
inv_lower    3           28.0  36.199286  11.648150  17.74  25.3575  40.410   
             6           23.0  34.823043   8.843265  25.40  27.4950  28.020   
lower        3           28.0  43.064286  10.094556  25.11  39.3675  41.905   
             6           25.0  44.154800   7.577394  25.11  40.0800  41.560   

                            75%    max  
tems_lm_name tems_lm_n                  
inv_lower    3          43.5925  56.76  
             6          42.3750  50.81  
lower        3          50.8500  57.81  
             6          49.9200  58.18

In [60]:
df_all.groupby(['txs_lm_name', 'txs_lm_n']).bleu.describe()

count       mean        std    min      25%     50%  \
txs_lm_name txs_lm_n                                                        
inv_lower   3          26.0  38.853077   7.729340  25.11  38.8225  40.285   
            6          26.0  33.186154   9.356853  17.80  25.4450  27.820   
lower       3          27.0  42.979630  11.171130  17.74  38.7550  43.380   
            6          25.0  43.628400  10.188479  24.94  40.2200  43.240   

                          75%    max  
txs_lm_name txs_lm_n                  
inv_lower   3         42.3700  57.77  
            6         41.9925  56.93  
lower       3         50.7650  58.18  
            6         50.5500  57.16

In [61]:
df_all.groupby('referrer').bleu.describe()

,count,mean,std,min,25%,50%,75%,max
referrer,,,,,,,,
counter,47.0,40.484894,10.929620,25.11,27.850,41.920,43.61,58.18
inv_counter,9.0,31.513333,8.663550,17.80,25.110,34.000,38.16,43.45
preprocess_so,48.0,40.370208,9.749184,17.74,39.075,40.725,49.74,56.93


## 5size

In [62]:
df_5.groupby('dp_scorer').bleu.describe()

NameError: name 'df_5' is not defined

In [57]:
df_5.groupby('sa_scorer').bleu.describe()

,count,mean,std,min,25%,50%,75%,max
sa_scorer,,,,,,,,
inv_ltr_lasso,225.0,28.954400,9.065081,13.95,22.89,26.70,36.08,52.81
ltr_lasso,225.0,28.792222,8.684845,13.85,22.45,27.66,35.85,48.07
random,225.0,29.268889,8.946539,13.93,22.90,28.10,36.36,51.46


In [58]:
df_5.groupby(['tems_lm_name', 'tems_lm_n']).bleu.describe()

count       mean       std    min    25%    50%  \
tems_lm_name tems_lm_n                                                    
inv_lower    3          135.0  23.047407  6.155775  13.93  19.35  22.79   
             6          135.0  23.638370  6.496942  13.85  19.78  23.39   
lower        3          135.0  35.073037  7.892178  22.47  27.74  37.02   
             6          135.0  34.327630  7.874061  21.76  27.29  35.79   
random       0          135.0  28.939407  7.914815  17.07  24.53  27.66   

                           75%    max  
tems_lm_name tems_lm_n                 
inv_lower    3          26.915  40.04  
             6          27.525  40.76  
lower        3          40.570  52.81  
             6          39.430  52.28  
random       0          34.650  48.95

In [59]:
df_5.groupby(['txs_lm_name', 'txs_lm_n']).bleu.describe()

count       mean       std    min     25%    50%  \
txs_lm_name txs_lm_n                                                     
inv_lower   3         135.0  24.966593  7.316950  13.85  20.985  23.13   
            6         135.0  25.214000  7.415394  14.10  21.205  23.25   
lower       3         135.0  33.671852  8.811930  17.59  27.360  32.39   
            6         135.0  33.362296  8.651870  17.73  27.370  32.17   
random      0         135.0  27.811111  7.937564  15.01  23.050  25.23   

                         75%    max  
txs_lm_name txs_lm_n                 
inv_lower   3         33.395  37.85  
            6         33.865  38.35  
lower       3         40.470  52.81  
            6         39.965  52.69  
random      0         35.425  44.04

# Lasso

In [9]:
df_all = df_all.drop(columns=['meteor', 'ter'])

In [15]:
from sklearn.linear_model import Lasso

In [37]:
X = df_all.drop(columns=['bleu', 'max_dp', 'max_sa', 'max_tems'])
y = df_all[['bleu']]

X_ohe = pd.get_dummies(X, columns=X.columns)

In [38]:
lasso = Lasso().fit(X_ohe, y)

In [39]:
lasso.coef_

array([ 0.        , -0.        ,  0.        , -0.        ,  0.        ,
       -0.        ,  0.        ,  0.        , -3.56075308,  3.79845913,
       -0.        , -0.        ,  0.        , -0.        ,  0.38945086,
       -9.21999944,  0.        , -1.13690988,  4.7128114 , -0.        ,
       -0.        ,  0.        , -0.        ])

In [50]:
result = pd.Series(data=lasso.coef_, index=X_ohe.columns)
result.sort_values(ascending=False)

txs_lm_name_lower          4.712811
tems_lm_name_lower         3.798459
referrer_counter           0.389451
txs_lm_n_6                -0.000000
tems_lm_name_random       -0.000000
dp_scorer_inv_ltr_lasso   -0.000000
dp_scorer_ltr_lasso        0.000000
dp_scorer_random          -0.000000
sa_scorer_gold             0.000000
sa_scorer_inv_ltr_lasso   -0.000000
sa_scorer_ltr_lasso        0.000000
sa_scorer_random           0.000000
tems_lm_n_0               -0.000000
txs_lm_n_3                 0.000000
tems_lm_n_3                0.000000
tems_lm_n_6               -0.000000
referrer_preprocess_so     0.000000
txs_lm_name_random        -0.000000
txs_lm_n_0                -0.000000
dp_scorer_gold             0.000000
txs_lm_name_inv_lower     -1.136910
tems_lm_name_inv_lower    -3.560753
referrer_inv_counter      -9.219999
dtype: float64